# Étape 1 – Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


# Étape 2 – Chargement du jeu de données

In [ ]:
df = pd.read_csv("dataset.csv")
df.head()

# Étape 3 - Analyse de la nature des colonnes

| Colonne               | Nature     |
|-----------------------|------------|
| nom                   | catégoriel |
| prenom                | catégoriel |
| age                   | ordinal    |
| taille                | ordinal    |
| poids                 | ordinal    |
| sexe                  | catégoriel |
| sport_licence         | catégoriel |
| niveau_etude          | ordinal    |
| region                | catégoriel |
| smoker                | catégoriel |
| nationalité_francaise  | catégoriel |
| revenu_estime_mois    | ordinal    |
| situation_familiale   | catégoriel |
| historique_credits    | ordinal    |
| risque_personnel      | ordinal    |
| date_creation_compte  | ordinal    |
| score_credit          | ordinal    |
| loyer_mensuel         | ordinal    |
| montant_pret          | ordinal    |


# Étape 4 – Analyse des valeurs manquantes

In [ ]:
# Suppression des doublons
df = df.drop_duplicates()
# Colonnes avec des valeurs manquantes
df.isnull().sum().sort_values(ascending=False)


In [ ]:
# Visualisation avec missingno
msno.matrix(df, figsize=(12,6))
msno.heatmap(df)

On constate que les colonnes score_credit, historique_credits, loyer_mensuel et situation_familiale  possèdent beaucoup de valeurs à null , il faudra palier à ce problème pour pourvoir exploiter les données.


# Étape 5 - Détection des outliers:

Afin de s'assurer que notre dataset ne possède pas de données "isolées" pour l'entrainement du futur modèle, nous devons détecter les outliers et les adapter en conséquence.

Ce code calcule l’intervalle interquartile (IQR) pour une colonne, définit des bornes (Q1 − 1.5×IQR, Q3 + 1.5×IQR), et retourne les lignes du DataFrame qui sortent de cet intervalle (outliers).

In [ ]:
# Fonction IQR pour détecter les outliers
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1 # mesure la dispersion au centre (intervalle contenant les 50 % du milieu).
    lower_bound = Q1 - 1.5 * IQR # seuil minimal accepté.
    upper_bound = Q3 + 1.5 * IQR # seuil maximal accepté.
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

# ➤ 3. Détection et traitement des outliers avec IQR (remplacement par borne)
colonnes_numeriques = df.select_dtypes(include=["float", "int"]).columns

for column in colonnes_numeriques:
    outliers = detect_outliers(df, column)
outliers.head()

In [ ]:
for column in colonnes_numeriques:
    # Boxplot
    plt.figure(figsize=(10,4))
    sns.boxplot(data=df, x=column)
    plt.title("Boxplot de la variable '"+column+"'")
    plt.draw()

Comme on peut constater sur les différents graphiques, il y aura des outliers à rectifier

# Étape 6 - Détection de biais en comparant les données :

In [ ]:
# Conversion des variables catégorielles en variables numériques avec "Label Encoding"
df['niveau_etude'] = df['niveau_etude'].map({
    'aucun': 0,
    'bac': 1,
    'bac+2': 2,
    'master': 3,
    'doctorat': 4
})

sns.pairplot(df, hue="niveau_etude", diag_kind="kde")

In [ ]:
pivot = df.pivot_table(index="sexe", columns="niveau_etude", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot,annot=True)

Ici, on peut constater que le montant de prêt est le plus élevé chez les personnes qui ont un niveau d'étude à Bac+5 et plus

### Détection de biais entre la catégorie fumeur et le montant du pret :

In [ ]:
sns.pairplot(df, hue="smoker", diag_kind="kde")

In [ ]:
pivot = df.pivot_table(index="smoker", columns="situation_familiale", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot)

On constate ici que les personnes non-fumeuses on tendance à obtenir un prêt plus important que celles qui fument.

### Detection de biais entre la nationalité et le montant du prêt :

In [ ]:
sns.pairplot(df, hue="nationalité_francaise", diag_kind="kde")

In [ ]:
pivot = df.pivot_table(index="nationalité_francaise", columns="situation_familiale", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot)

In [ ]:
print(df["nationalité_francaise"].value_counts())

On constate ici que les personnes qui ont une nationnalité française ont tendance à avoir un prêt plsu important que celles qui ne le sont pas.

### Biais détecté entre concernant le sexe, la situation familiale et  montant pret :

In [ ]:
pivot = df.pivot_table(index="sexe", columns="situation_familiale", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot)

On constate ici que les personnes qui ont une nationnalité française ont tendance à avoir un prêt plus important que celles qui ne le sont pas.

### Biais detecté sur la région , le sexe et le montant pret d'une personne :

In [ ]:
pivot = df.pivot_table(index="sexe", columns="region", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot)

On constate ici que les personnes qui se situent en île de France, obtiennent un montant de prêt plus important ce qui créé un biais à ce niveau là

## Biais détecté sur la licence de sport

In [ ]:
pivot = df.pivot_table(index="sport_licence", columns="situation_familiale", values="montant_pret", aggfunc="mean")
sns.heatmap(pivot)

On constate qu'il y a un biais entre le montant pret et la possession d'une licence de sport, le montant du pret à tandance à être plus élevé si la personne possède une licence

### Tableau de corrélations

In [ ]:
df_new = df.copy()
df_new['niveau_etude'] = df['niveau_etude'].map({
    'aucun': 0,
    'bac': 1,
    'bac+2': 2,
    'master': 3,
    'doctorat': 4
})
df_new['sexe'] = df['sexe'].map({
    'H': 1, # H : 0 et F: 1 auraient également fait l'affaire pour ce dont on a besoin de montrer
    'F': 0
})
df_new['smoker'] = df['smoker'].map({
    'oui': 1,
    'non': 0
})
df_new['sport_licence'] = df['sport_licence'].map({
    'oui': 1,
    'non': 0
})
df_new['nationalité_francaise'] = df['nationalité_francaise'].map({
    'oui': 1,
    'non': 0
})


df_new['situation_familiale'] = df['situation_familiale'].map({
    "célibataire" : 5,
    "divorcé" : 10,
    "veuf" : 7 ,
    "marié" : 10
})

df_new['region'] = df['region'].map({
    "Occitanie" : 0,
    "Île-de-France" : 1,
    "Auvergne-Rhône-Alpes" : 2,
    "Corse" : 3,
    "Bretagne" : 4,
    "Hauts-de-France" : 5,
    "Provence-Alpes-Côte d’Azur" : 6,
    "Normandie" : 7
})
plt.figure(figsize=(10,4))
sns.heatmap(df_new.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")

Sur le tableau de corrélation ci-dessus on constate clairement que les colonnes `sexe`, `sport_licence`, `niveau_etude`, `region`, `smoker`, `nationalité_francaise`, `revenu_estime_mois`, `score_credit` ont une influence sur la valeur de `montant_pret`.

_Note: On a volontairement "converti" les données catégorielles pour qu'elles soient numériques (par exemple H = 1 et F= 0 mais on aurait bien pu changer ses valeurs , F = 1 et H = 0 va juste rendre le nombre en retour positif), le but ici n'est pas de voir la valeur de la corrélation en soi mais de comprendre qu'il y a une influence sur le montant de pret comme l'a montré l'analyse précédente_

# Etape 7 - Nettoyage et traitement

## Point de vue Ethique :

Suite à l'analyse précédente et pour des questions éthiques, nous exclurons les colonnes suivantes :
- `nom`, `prenom` : car il s'agit de données identifiantes
- `smoker` : un biais a été relevé dans notre analyse sur la colonne `montant_pret`, de plus, peut être assimilé à une donnée santé, de plus,
- `region`, `sexe`, `nationalité_francaise`, `niveau_etude` : des biais ont été identifiés dans l'analyse précédente
- `taille` , `poids` , `age` : ces données n'ont pas spécialement de corrélations avec le montant pret, on peut les écarter du dataset

## Autres colonnes à retirer :

N'ayant pas constaté de corrélation directe concernant les colonnes `date_creation_compte` et `situation_familiale` nous allons les retirer également

Les données financières telles que : `historique_credits`, `risque_personnel`, `score_credit`, `revenu_estime_mois` seront gardées pour permettre une meilleure estimation du montant prêt par le futur modèle


In [ ]:
df = df.drop(columns=["nom","prenom","sexe", "sport_licence", "niveau_etude", "region", "smoker", "nationalité_francaise","taille" , "poids" , "age", "situation_familiale", "date_creation_compte"])

### 1. Suppression des colonnes quasi-vides (plus de 90% de valeurs manquantes)

In [ ]:
seuil_colonnes_vides = 0.90
colonnes_a_supprimer = df.columns[df.isnull().mean() > seuil_colonnes_vides]
df.drop(columns=colonnes_a_supprimer, inplace=True)

print(f"Colonnes supprimées (>{int(seuil_colonnes_vides*100)}% de valeurs manquantes) : {list(colonnes_a_supprimer)}")

### 2. Suppression des lignes trop incomplètes (plus de 50% de valeurs nulles ou selon seuil)

In [ ]:
# ➤ 2. Suppression des lignes trop incomplètes (plus de 50% de valeurs nulles ou selon seuil)
seuil_lignes = 0.5
lignes_avant = df.shape[0]
df = df[df.isnull().mean(axis=1) < seuil_lignes]
lignes_apres = df.shape[0]

print(f"Lignes supprimées : {lignes_avant - lignes_apres} (incomplètes à plus de {int(seuil_lignes*100)}%)")

### 3. Détection et traitement des outliers avec IQR (remplacement par borne)

In [ ]:
colonnes_numeriques = df.select_dtypes(include=["float", "int"]).columns

for col in colonnes_numeriques:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)][col]

    if not outliers.empty:
        df[col] = np.where(df[col] < lower, lower, df[col])
        df[col] = np.where(df[col] > upper, upper, df[col])
        print(f"{col} : {len(outliers)} outliers remplacés par bornes IQR")

In [ ]:
# Visualisation avec missingno
msno.matrix(df, figsize=(12,6))
# Colonnes avec des valeurs manquantes
df.isnull().sum().sort_values(ascending=False)

### 4. Imputation des valeurs manquantes selon stratégie par variable


In [ ]:
from sklearn.impute import KNNImputer
colonnes_numeriques_peu_correlees= ["historique_credits","loyer_mensuel"]
colonnes_numeriques_correlees= ["revenu_estime_mois", "montant_pret", "score_credit"]
# Remplissage par médiane pour les colonnes numériques peu corrélées
for col in colonnes_numeriques_peu_correlees:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())
        print(f"{col} : valeurs nulles imputées par médiane")

# Exemple avec KNNImputer pour certaines colonnes numériques corrélées
imputer = KNNImputer(n_neighbors=5)
df[colonnes_numeriques_correlees] = imputer.fit_transform(
    df[colonnes_numeriques_correlees]
)
print("Imputation par KNN effectuée sur : ")
print(colonnes_numeriques_correlees)

# Visualisation avec missingno
msno.matrix(df, figsize=(12,6))
# Colonnes avec des valeurs manquantes
df.isnull().sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,4))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")

Apres imputation des valeurs, nous conservons les bonnes corrélations pour les colonnes `revenu_estime_mois` , `score_credit`

In [ ]:
#Résumé statistique après nettoyage
df.describe().T

# Etape 8 - Génération du dataset nettoyé

In [ ]:
df.to_csv('dataset_clean_ethics.csv', index=False)